# importing the basic libraries

In [24]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder

import spacy

import warnings
warnings.filterwarnings('ignore')


In [2]:
df=pd.read_csv('twitter_training.csv',names = ['id','country','Label','Text'])

In [3]:
df.head()

,id,country,Label,Text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [4]:
df.head()

,id,country,Label,Text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [5]:
df.isnull().sum()

,0
id,0
country,0
Label,0
Text,686


In [6]:
# demoving the null values from the dataset
df=df.dropna()

In [7]:
df.isnull().sum()

,0
id,0
country,0
Label,0
Text,0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 73996 entries, 0 to 74681
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       73996 non-null  int64 
 1   country  73996 non-null  object
 2   Label    73996 non-null  object
 3   Text     73996 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.8+ MB


In [9]:
df.shape

(73996, 4)

In [10]:
df['Label'].value_counts()


,count
Label,
Negative,22358
Positive,20655
Neutral,18108
Irrelevant,12875


# Preprocessing the text data according to the sentement analysis

In [11]:
! pip install autocorrect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622364 sha256=19535304b1799b8866aad6a0630fcbed6e679bd4c9a9a23c1c953f2980ede867
  Stored in directory: /root/.cache/pip/wheels/b5/7b/6d/b76b29ce11ff8e2521c8c7dd0e5bfee4fb1789d76193124343
Successfully built autocorrect


In [12]:
# import re
# import nltk
# from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer
# from nltk.tokenize import word_tokenize
# from autocorrect import Speller  # For spelling correction

# # Download necessary NLTK data
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('omw-1.4')
# '''
# 1. nltk.download('punkt'): Downloads the Punkt Tokenizer Models, which are used for tokenizing text (splitting text into individual words or tokens).

# 2. nltk.download('stopwords'): Downloads the Stopwords corpus, which contains a list of common words like "the", "and", "a", etc. that do not carry much meaning in a sentence and are often removed in NLP tasks.

# 3. nltk.download('wordnet'): Downloads the WordNet corpus, which is a large lexical database of English words and their relationships (synonyms, antonyms, hyponyms, etc.).

# 4. nltk.download('omw-1.4'): Downloads the Open Multilingual WordNet (OMW) dataset, which is a multilingual wordnet (a lexical database) that includes words and their translations in many languages.
# '''

# # Sample text
# text = "This is an example of Text Normalization in NLP! It includes numbers like 123, punctuation, contractions like don't, and spelling mistakes like 'hte' instead neme of 'the'."
# def text_normalization(text):


#     # 1. Lowercasing
#     text = text.lower()

#     # 2. Removing Punctuation
#     text = re.sub(r'[^\w\s]', '', text)

#     # 3. Tokenization
#     tokens = word_tokenize(text)

#     # 4. Removing Stop Words
#     stop_words = set(stopwords.words('english'))
#     tokens = [word for word in tokens if word not in stop_words]

#     # 5. Lemmatization
#     lemmatizer = WordNetLemmatizer()
#     tokens = [lemmatizer.lemmatize(word) for word in tokens]

#     # 6. Stemming (using lemmatization instead for this case as it is more accurate)
#     # Stemming would be: from nltk.stem import PorterStemmer
#     # stemmer = PorterStemmer()
#     # tokens = [stemmer.stem(word) for word in tokens]

#     # 7. Removing Numbers
#     tokens = [word for word in tokens if not word.isdigit()]

#     # 8. Removing Special Characters (already covered by punctuation removal)
#     # This step is handled in step 2 (removing punctuation) but can also remove special symbols.

#     # 9. Spelling Correction (using the autocorrect library)
#     spell = Speller()
#     tokens = [spell(word) for word in tokens]

#     # 10. Expanding Contractions (manual expansion or using contractions library)
#     contractions_dict = {"dont": "do not", "wont": "will not"}
#     tokens = [contractions_dict.get(word, word) for word in tokens]

#     # 11. Removing Extra Whitespace (handled by tokenization, which removes space between words)

#     # 12 Joining the tokens back into a string
#     normalized_text = ' '.join(tokens)

#     return normalized_text


In [13]:
# text_normalization(text)

In [14]:
df1=df.copy()

In [15]:
# df1['Preprocessed Text'] = df1['Text'].apply(text_normalization)

In [16]:
# Preprocess Function
nlp = spacy.load("en_core_web_sm")
def preprocess(text):
    doc = nlp(text)
    filtered_tokens = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_tokens.append(token.lemma_)

    return " ".join(filtered_tokens)

In [17]:
df['Preprocessed Text'] = df['Text'].apply(preprocess)

In [18]:
df.head()

,id,country,Label,Text,Preprocessed Text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...,m get borderland murder
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...,come border kill
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...,m get borderland kill
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...,m come borderland murder
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...,m get borderland 2 murder


In [19]:
# Encoding target column
le = LabelEncoder()
df['Label'] = le.fit_transform(df['Label'])

In [20]:
df.head()

,id,country,Label,Text,Preprocessed Text
0,2401,Borderlands,3,im getting on borderlands and i will murder yo...,m get borderland murder
1,2401,Borderlands,3,I am coming to the borders and I will kill you...,come border kill
2,2401,Borderlands,3,im getting on borderlands and i will kill you ...,m get borderland kill
3,2401,Borderlands,3,im coming on borderlands and i will murder you...,m come borderland murder
4,2401,Borderlands,3,im getting on borderlands 2 and i will murder ...,m get borderland 2 murder


In [21]:
# splite the data into trainm and test
X_train, X_test, y_train, y_test = train_test_split(df['Preprocessed Text'], df['Label'], test_size=0.2, random_state=42)

In [22]:
X_train.shape

(59196,)

In [23]:
X_test.shape

(14800,)

# featur selection

In [26]:
# Import the necessary class
from sklearn.feature_extraction.text import CountVectorizer

# Initialize CountVectorizer
te = CountVectorizer()

# Fit and transform the training data to get word counts
X_train_counts = te.fit_transform(X_train)

# Transform the test data using the fitted CountVectorizer
X_test_counts = te.transform(X_test)

# Initialize TfidfTransformer
from sklearn.feature_extraction.text import TfidfTransformer # Import TfidfTransformer here

tfidf_transformer = TfidfTransformer()

# Fit and transform the word counts to TF-IDF representation
# Use X_train_counts instead of X_train
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

# Transform the test data using the fitted TfidfTransformer
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

In [27]:
y_train.value_counts()

,count
Label,
1,17978
3,16536
2,14503
0,10179


In [28]:
from sklearn.preprocessing import StandardScaler,LabelEncoder
scaler = StandardScaler(with_mean=False)
X_train_tfidf = scaler.fit_transform(X_train_tfidf)
X_test_tfidf = scaler.transform(X_test_tfidf)

# model building

# Logistic regration

In [29]:
from sklearn.linear_model import LogisticRegression
model_l=LogisticRegression()
model_l.fit(X_train_tfidf,y_train)

LogisticRegression()

In [30]:
y_pred=model_l.predict(X_test_tfidf)

In [31]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.86      0.80      0.83      2696
           1       0.86      0.85      0.85      4380
           2       0.86      0.82      0.84      3605
           3       0.79      0.86      0.82      4119

    accuracy                           0.84     14800
   macro avg       0.84      0.83      0.84     14800
weighted avg       0.84      0.84      0.84     14800



In [32]:
import pickle
filename = 'sentiment_analysis_model.pkl'
pickle.dump(model_l, open(filename, 'wb'))

# Random forest

In [ ]:
# using the random forest model here
rf_model = RandomForestClassifier()
rf_model.fit(X_train_tfidf, y_train)

RandomForestClassifier()

In [ ]:
rf_pred=rf_model.predict(X_test_tfidf)
print(classification_report(y_test,rf_pred))

              precision    recall  f1-score   support

           0       0.97      0.84      0.90      2696
           1       0.92      0.93      0.92      4380
           2       0.94      0.89      0.91      3605
           3       0.85      0.94      0.89      4119

    accuracy                           0.91     14800
   macro avg       0.92      0.90      0.91     14800
weighted avg       0.91      0.91      0.91     14800



In [ ]:
# here using the byes algorith
byes_model = MultinomialNB()
byes_model.fit(X_train_tfidf, y_train)

MultinomialNB()

In [ ]:
byes_model_pred=byes_model.predict(X_test_tfidf)
print(classification_report(y_test,byes_model_pred))

              precision    recall  f1-score   support

           0       0.83      0.75      0.79      2696
           1       0.74      0.87      0.80      4380
           2       0.90      0.74      0.81      3605
           3       0.78      0.81      0.79      4119

    accuracy                           0.80     14800
   macro avg       0.81      0.79      0.80     14800
weighted avg       0.81      0.80      0.80     14800



In [ ]:
from sklearn.svm import SVC

# Support Vector Machine (SVM) model
svm_model = SVC()
svm_model.fit(X_train_tfidf, y_train)
svm_pred = svm_model.predict(X_test_tfidf)
print(classification_report(y_test, svm_pred))


In [ ]:
# load ml model

loaded_model = pickle.load(open('sentiment_analysis_model.pkl', 'rb'))


In [ ]:
y=loaded_model.predict(X_train_tfidf)

In [ ]:
print(classification_report(y_train,y))

# load the ml model for sentement analysis

In [ ]:
# load ml model here
import pickle
filename = 'sentiment_analysis_model.pkl'
loaded_model = pickle.load(open(filename, 'rb'))


In [ ]:
# Preprocess Function
nlp = spacy.load("en_core_web_sm")
def preprocess(text):
    doc = nlp(text)
    filtered_tokens = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_tokens.append(token.lemma_)

    return " ".join(filtered_tokens)

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
sentinel_data['label'] = label_encoder.fit_transform(sentinel_data['label'])

In [ ]:
# Split the data into the train and test fir mal model trainingn and teting prpuss
X_train, X_test, y_train, y_test = train_test_split(sentinel_data['tweet'], sentinel_data['label'], test_size=0.2, random_state=42)

# preprocesing the text data by uning the nltk and scapy

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
import re
def text_preprocessing(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # remove spacial chracter
    text = re.sub(r'\W', ' ', text)
    # Remove numbers
    text = re.sub(r'\d', '', text)
    # Remove single characters
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    #Preprocess the text data (e.g., remove stop words, stemming/lemmatization, etc.
    token=



In [ ]:


# Convert labels to numerical values (optional but often helpful for modeling)
label_mapping = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
sentinel_data['label'] = sentinel_data['label'].map(label_mapping)

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X = sentinel_data['tweet']
y = sentinel_data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocess the text data (e.g., remove stop words, stemming/lemmatization, etc.)
# You can use libraries like NLTK or spaCy for this.

# Feature extraction (e.g., using TF-IDF or word embeddings)
# You can use scikit-learn or other libraries for this.

# Train a machine learning model (e.g., Naive Bayes, Logistic Regression, SVM, etc.)
# You can use scikit-learn or other libraries for this.

# Evaluate the model's performance on the test set
# You can use metrics like accuracy, precision, recall, F1-score, etc.
